In [1]:
dbutils.fs.ls('/FileStore/tables/pc5ly1131502251351157/italianPosts.csv')

#### italianPosts.csv
1. commentCount—Number of comments related to the question/answer
1. lastActivityDate—Date and time of the last modification
1. ownerUserId—User ID of the owner
1. body—Textual contents of the question/answer
1. score—Total score based on upvotes and downvotes
1. creationDate—Date and time of creation
1. viewCount—View count
1. title—Title of the question
1. tags—Set of tags the question has been marked with
1. answerCount—Number of related answers
1. acceptedAnswerId—If a question contains the ID of its accepted answer
1. postTypeId—Type of the post; 1 is for questions, 2 for answers
1. id—Post’s unique ID

In [3]:
# load into rdd
rddItalianPosts_1 = sc.textFile('/FileStore/tables/pc5ly1131502251351157/italianPosts.csv') \
                    .map(lambda line: line.split('~'))
rddItalianPosts_1.count()
rddItalianPosts_1.take(1)

#### 5.1.1
#### Creating a DataFrame from an RDD of tuples

In [5]:
# convert each RDD array to tuple
rddItalianPosts_2 = rddItalianPosts_1.map(lambda l: (l[0], l[1], l[2], l[3], l[4], l[5], l[6], l[7], l[8], l[9], l[10], l[11], l[12]))
rddItalianPosts_2.take(1)

In [6]:
# now convert to DF
dfItalianPosts = rddItalianPosts_2.toDF(['commentCount', 'lastActivityDate', 'ownerUserId', 'body', 'score', 'creationDate', 'viewCount', 'title', 'tags', 'answerCount', 'acceptedAnswerId', 'postTypeId', 'id'])
dfItalianPosts.show(2)

##### Note that the field data type are incorrect

In [8]:
dfItalianPosts.printSchema()

We'll need to define schema manually to set the proper datatypes

#### Converting RDDs to DataFrames by specifying a schema

In [11]:
from datetime import datetime

def toIntSafe(val):
  try:
    return int(val)
  except ValueError:
    return None
  
def toLongSafe(val):
  try:
    return long(val)
  except ValueError:
    return None
  
def toTimeSafe(val):
  try:
    return datetime.strptime(val, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None
  
# Method to convert string to a Row
from pyspark.sql import Row
def stringToPost(string):
  l_s = string.encode('utf8').strip().split('~')
  return Row(
    toIntSafe(l_s[0]),   # commentCount
    toTimeSafe(l_s[1]),  # lastActivityDate
    toLongSafe(l_s[2]),  # ownerUserId
    l_s[3],              # body
    toIntSafe(l_s[4]),   # score
    toTimeSafe(l_s[5]),  # creationDate
    toIntSafe(l_s[6]),   # viewCount
    l_s[7],              # title
    l_s[8],              # tags
    toIntSafe(l_s[9]),   # answerCount
    toLongSafe(l_s[10]), # acceptedAnswerId
    toLongSafe(l_s[11]), # postTypeId
    toLongSafe(l_s[12]), # id
  )
  
# Define schema
from pyspark.sql.types import *
postSchema = StructType([
    StructField('commentCount', IntegerType(), True),
    StructField('lastActivityDate', TimestampType(), True),
    StructField('ownerUserId', LongType(), True),
    StructField('body', StringType(), True),
    StructField('score', IntegerType(), True),
    StructField('creationDate', TimestampType(), True),
    StructField('viewCount', IntegerType(), True),
    StructField('title', StringType(), True),
    StructField('tags', StringType(), True),
    StructField('answerCount', IntegerType(), True),
    StructField('acceptedAnswerId', LongType(), True),
    StructField('postTypeId', LongType(), True),
    StructField('id', LongType(), False),
  ])

In [12]:
# Now, create RDD using schema
rddItalianPosts_3 = sc.textFile('/FileStore/tables/pc5ly1131502251351157/italianPosts.csv') \
                      .map(lambda line: stringToPost(line))
rddItalianPosts_3.count()

In [13]:
dfItalianPosts_1 = sqlContext.createDataFrame(rddItalianPosts_3, schema=postSchema)
dfItalianPosts_1.take(1)

In [14]:
dfItalianPosts_1.printSchema()

In [15]:
dfItalianPosts_1.columns

In [16]:
dfItalianPosts_1.dtypes

### 5.1.2 API basics

##### [select](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select)

In [19]:
# 1. select can take column names
dfPostsIdBody = dfItalianPosts_1.select('id', 'body')
dfPostsIdBody.show(1)

In [20]:
# 2. select can take Column type
dfPostsIdBody = dfItalianPosts_1.select(dfItalianPosts_1.id, dfItalianPosts_1.body) # this is the most used methon in spark doc
dfPostsIdBody = dfItalianPosts_1.select(dfItalianPosts_1['id'], dfItalianPosts_1['body'])

from pyspark.sql.functions import col
dfPostsIdBody = dfItalianPosts_1.select(col('id'), col('body'))
dfPostsIdBody.show(1)

In [21]:
dfPostsExceptBody = dfItalianPosts_1.select('*').drop(dfItalianPosts_1.body)
dfPostsExceptBody.show(1)

In [22]:
# where() is an alias of filter()
# dfPostsIdBody.filter('Italiano' in dfPostsIdBody.body).count()
from pyspark.sql.functions import instr
dfPostsIdBody.filter(instr(dfPostsIdBody.body, 'Italiano') > 0).count()

In [23]:
# NOTE how '&' is used for 'and', parenthesis around each sub expression 
dfPostsExceptBody.filter((dfPostsExceptBody.postTypeId == 1) & (dfPostsExceptBody.acceptedAnswerId.isNull())).count()

In [24]:
# limit the result set
dfPostsExceptBody.filter((dfPostsExceptBody.postTypeId == 1) & (dfPostsExceptBody.acceptedAnswerId.isNull())).limit(10).show()

In [25]:
dfPostsExceptBody.withColumnRenamed('ownerUserId', 'ownerId').show(1)

In [26]:
dfPostsExceptBody.withColumn('ratio', dfPostsExceptBody.viewCount / dfPostsExceptBody.score).filter('ratio < 35').count()

In [27]:
# The 10 most recently modified questions
dfPostsExceptBody.orderBy(dfPostsExceptBody.lastActivityDate.desc()).limit(10).show()

In [28]:
dfPostsExceptBody.orderBy(dfPostsExceptBody.lastActivityDate, ascending=False).limit(10).show()

In [29]:
dfPostsExceptBody.orderBy('lastActivityDate', ascending=False).limit(10).show()

In [30]:
from pyspark.sql.functions import desc
dfPostsExceptBody.orderBy(desc('lastActivityDate')).limit(10).show()

In [31]:
# multiple sort orders
dfPostsExceptBody.orderBy([dfPostsExceptBody.lastActivityDate, dfPostsExceptBody.creationDate], ascending=[False, True]).limit(10).show()

### 5.1.3 SQL Functions

SQL functions fit into four categories:
* Scalar functions return a single value for each row based on calculations on one or more columns.
* Aggregate functions return a single value for a group of rows.
* Window functions return several values for a group of rows.
* User-defined functions include custom scalar or aggregate functions.

In [33]:
# find the question that was active for the largest amount of time
from pyspark.sql.functions import *
dfItalianPosts_1 \
  .filter(dfItalianPosts_1.postTypeId == 1) \
  .withColumn('activePeriod', datediff(dfItalianPosts_1.lastActivityDate, dfItalianPosts_1.creationDate)) \
  .orderBy(desc('activePeriod')) \
  .head().body.replace('&lt;', '<').replace('&gt;', '>')

In [34]:
# find the average and maximum score of all questions and the total number of questions
dfItalianPosts_1.select(avg(dfItalianPosts_1.score), max(dfItalianPosts_1.score), count(dfItalianPosts_1.score)).show()

In [35]:
## First, you’ll display the maximum score of all user questions (post typeID 1),
## and for each question, how much its score is below the maximum score for that user.

from pyspark.sql.functions import *
from pyspark.sql.window import *
dfIP_1 = dfItalianPosts_1 \
  .filter(dfItalianPosts_1.postTypeId == 1) \
  .select(dfItalianPosts_1.ownerUserId, dfItalianPosts_1.acceptedAnswerId, dfItalianPosts_1.score, max(dfItalianPosts_1.score).over(Window.partitionBy(dfItalianPosts_1.ownerUserId)).alias('maxPerUser'))
dfIP_1.show()

In [36]:
dfIP_2 = dfIP_1.withColumn('toMax', dfIP_1.maxPerUser - dfIP_1.score)
dfIP_2.show()

In [37]:
# For each question, display the id of its owner’s next and previous questions by creation date

from pyspark.sql.functions import *
from pyspark.sql.window import *
dfIP_1 = dfItalianPosts_1 \
  .filter(dfItalianPosts_1.postTypeId == 1) \
  .select(dfItalianPosts_1.ownerUserId, dfItalianPosts_1.id, dfItalianPosts_1.creationDate,\
          lag(dfItalianPosts_1.id, count=1).over(Window.partitionBy(dfItalianPosts_1.ownerUserId).orderBy(dfItalianPosts_1.creationDate)).alias('prev'),\
          lead(dfItalianPosts_1.id, count=1).over(Window.partitionBy(dfItalianPosts_1.ownerUserId).orderBy(dfItalianPosts_1.creationDate)).alias('next')) \
  .orderBy([dfIP_1.ownerUserId, dfItalianPosts_1.id])
dfIP_1.show()

#### UDF: User-defined functions
UDF's let us extend build-in sql.functions

In [39]:
# 1. Use udf() to create a UDF
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
countTags = udf(lambda tags: tags.count('&lt;'), IntegerType())
dfIP_1 = dfItalianPosts_1 \
  .filter(dfItalianPosts_1.postTypeId == 1) \
  .select(dfItalianPosts_1.tags, countTags(dfItalianPosts_1.tags).alias('tagCount'))
dfIP_1.show(10, False)

In [40]:
# 2. User SparkSession udf.register to register UDF
# This also allows us to use UDF in SQL expressions
from pyspark.sql.types import StringType

sqlContext.udf.register('replaceAngleBracketsUDF', lambda tags: tags.replace('&lt;', '<').replace('&gt;', '>'), StringType())

dfIP_1 = dfItalianPosts_1 \
  .filter(dfItalianPosts_1.postTypeId == 1) \
  .selectExpr('replaceAngleBracketsUDF(tags)')
dfIP_1.show(10, False)


### 5.1.4

In [42]:
# na returns DataFrameNaFunctions
dfClean = dfItalianPosts_1.na.drop() # drop rows if any column has null
dfClean.show()
dfClean.count()

In [43]:
dfItalianPosts_1.na.drop(subset=['acceptedAnswerId']).show()

In [44]:
dfItalianPosts_1.na.fill({'viewCount': 0}).show()

In [45]:
dfItalianPosts_1.na.replace(1177, 3000, ['id', 'acceptedAnswerId']).show()

### 5.1.5 Converting DataFrame to RDD

Typically there’s no need to convert DataFrames to RDDs and back because most
data-mapping tasks can be done with built-in DSL and SQL functions and UDFs.

In [48]:
def replaceLtGt(row):
  return Row(
    commentCount=row.commentCount,
    lastActivityDate=row.lastActivityDate,
    ownerUserId=row.ownerUserId,
    body=row.body.replace('&lt;', '<').replace('&gt;', '>'),
    score=row.score,
    creationDate=row.creationDate,
    viewCount=row.viewCount,
    title=row.title,
    tags=row.tags.replace('&lt;', '<').replace('&gt;', '>'),
    answerCount=row.answerCount,
    acceptedAnswerId=row.acceptedAnswerId,
    postTypeId=row.postTypeId,
    id=row.id
  )
rddPosts = dfItalianPosts_1.rdd.map(replaceLtGt)
rddPosts.take(1)

In [49]:
# RDD to DF with reordering columns
def sortSchema(schema):
  fields = {f.name: f for f in schema.fields}
  names = sorted(fields.keys())
  return StructType([fields[name] for name in names])

dfItalianPosts_mapped = sqlContext.createDataFrame(rddPosts, schema=sortSchema(dfItalianPosts_1.schema))
dfItalianPosts_mapped.show(5)

### 5.1.6. Grouping and joining data
DataFrame.groupBy() returns GroupedData()

In [51]:
# To find the number of posts per author, associated tags, and the post type
dfPostsPerAuthor = dfItalianPosts_mapped.groupBy(['ownerUserId', 'tags', 'postTypeId']).count()
dfPostsPerAuthor.orderBy(dfItalianPosts_mapped.ownerUserId.desc()).show(5)

In [52]:
# find the last activity date and the maximum post score per user
from pyspark.sql.functions import *
dfItalianPosts_mapped.groupBy(dfItalianPosts_mapped.ownerUserId) \
  .agg(max(dfItalianPosts_mapped.lastActivityDate), max(dfItalianPosts_mapped.score)) \
  .withColumnRenamed('max(lastActivityDate)', 'maxLastActivityDate') \
  .withColumnRenamed('max(score)', 'maxScore') \
  .orderBy(dfItalianPosts_mapped.ownerUserId) \
  .show(5)

In [53]:
dfItalianPosts_mapped.groupBy(dfItalianPosts_mapped.ownerUserId) \
  .agg({'lastActivityDate': 'max', 'score': 'max'}) \
  .orderBy('ownerUserId') \
  .show(5)

In [54]:
dfItalianPosts_mapped.groupBy(dfItalianPosts_mapped.ownerUserId) \
  .agg(max(dfItalianPosts_mapped.lastActivityDate), max(dfItalianPosts_mapped.score) > 5) \
  .orderBy('ownerUserId') \
  .show(5)

In [55]:
# rollup
dfSample = dfItalianPosts_mapped.where((dfItalianPosts_mapped.ownerUserId >= 13) & (dfItalianPosts_mapped.ownerUserId <= 15))
dfSample.count()

In [56]:
dfSample.show()

In [57]:
# Counting the posts by owner, tags, and post type
dfSample.groupBy(dfSample.ownerUserId, dfSample.tags, dfSample.postTypeId).count().show()

In [58]:
dfSample.rollup(dfSample.ownerUserId, dfSample.tags, dfSample.postTypeId).count().show()

In [59]:
dfSample.cube(dfSample.ownerUserId, dfSample.tags, dfSample.postTypeId).count().show()

#### Configuring Spark SQL
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession.conf

In [61]:
# SparkSession
spark.conf.set("SET spark.sql.caseSensitive=true")
spark.conf.set("spark.sql.caseSensitive", "true")

In [62]:
sqlContext.sql("SET spark.sql.caseSensitive=true")
sqlContext.setConf("spark.sql.caseSensitive", "true")

In [63]:
spark.conf.get('spark.sql.caseSensitive')

In [64]:
spark.conf.get('spark.sql.tungsten.enabled')

### 5.1.7 Performing Joins

In [66]:
dbutils.fs.ls('/FileStore/tables/buj2ar781502756511985/italianVotes.csv')

In [67]:
def italianVoteToRow(line):
  l_vote = line.encode('utf8').split('~')
  return Row(
    id=long(l_vote[0]),
    postId=long(l_vote[1]),
    voteTypeId=int(l_vote[2]),
    creationDate=datetime.strptime(l_vote[3], "%Y-%m-%d %H:%M:%S.%f") 
  )

rddItalianVotes = sc.textFile('/FileStore/tables/buj2ar781502756511985/italianVotes.csv').map(italianVoteToRow)
rddItalianVotes.take(5)

In [68]:
voteSchema = StructType([
    StructField('creationDate', TimestampType(), False),
    StructField('id', LongType(), False),
    StructField('postId', LongType(), False),
    StructField('voteTypeId', IntegerType(), False),
  ])

dfItalianVotes = sqlContext.createDataFrame(rddItalianVotes, schema=voteSchema)
dfItalianVotes.show(5)

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join

In [70]:
sqlContext.setConf('spark.sql.shuffle.partitions', 5)
dfItalianPosts_mapped.join(dfItalianVotes, dfItalianPosts_mapped.id == dfItalianVotes.postId, 'inner').show(5)

#### DataSets
The idea behind DataSets “is to provide an API that allows users to easily express transformations on domain objects, while
also providing the performance and robustness advantages of the Spark SQL execution engine”.
That essentially means that you can store ordinary Java objects in DataSets and take advantage of Tungsten and Catalyst optimizations.

DataFrames are now simply implemented as DataSets containing Row objects.

### 5.3.1

In [73]:
# Register temp table
dfItalianPosts_mapped.createOrReplaceTempView('posts_temp')

In [74]:
# Register table permanently
dfItalianVotes.write.saveAsTable('votes')

In [75]:
# View table cataloges
spark.catalog.listTables()

In [76]:
spark.catalog.listColumns('votes')

In [77]:
spark.catalog.listFunctions()

### 5.3.2 Executing SQL queries

In [79]:
dfPostsTemp = sqlContext.sql('SELECT * FROM posts_temp')
dfPostsTemp.show(5)

In [80]:
dfPostsTemp.printSchema()

In [81]:
%sql
select * from votes limit(10);

In [82]:
%sql
select substring(title, 0, 70) from posts_temp where postTypeId = 1 order by creationDate desc limit 3;